In [1]:
import pandas as pd
import plotly.express as px
import nbformat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor




In [2]:
Dinning = pd.read_excel("DinningData.xlsx", sheet_name='Data for Linear')


In [3]:
Dinning.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          240 non-null    int64  
 1   Semester     240 non-null    int64  
 2   Month        240 non-null    int64  
 3   Year         240 non-null    int64  
 4   Hours        205 non-null    float64
 5   Money Spent  240 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 11.4 KB


In [4]:
Dinning

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [5]:
missing_values = Dinning.isnull().sum()
missing_values


DEP             0
Semester        0
Month           0
Year            0
Hours          35
Money Spent     0
dtype: int64

In [6]:
print(Dinning.columns)

Index(['DEP', 'Semester', 'Month', 'Year', 'Hours', 'Money Spent'], dtype='object')


In [23]:


def impute_with_models(df, target_column, feature_columns, tolerance=0.1, n_neighbors=10,n_estimators=60, random_state=42):
    # Separate the data into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df.dropna(subset=[target_column])

    # If there are no missing values, return the original dataframe
    if df_missing.empty:
        print(f"No missing values for {target_column}. Skipping imputation.")
        return df, None, None, None, None, None, None, None, None  # Return None for metrics

    # Split df_not_missing into training and testing sets
    X = df_not_missing[feature_columns]
    y = df_not_missing[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit a linear regression model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the linear regression model
    lr_y_pred = lr_model.predict(X_test)

    # Calculate evaluation metrics for linear regression
    lr_mse = mean_squared_error(y_test, lr_y_pred)
    lr_mae = mean_absolute_error(y_test, lr_y_pred)
    lr_r2 = r2_score(y_test, lr_y_pred)

    print(f"Linear Regression Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {lr_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {lr_mae:.2f}")
    print(f"R-squared (R²): {lr_r2:.2f}")

    # Create and fit a KNN model
    knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the KNN model
    knn_y_pred = knn_model.predict(X_test)

    # Calculate evaluation metrics for KNN
    knn_mse = mean_squared_error(y_test, knn_y_pred)
    knn_mae = mean_absolute_error(y_test, knn_y_pred)
    knn_r2 = r2_score(y_test, knn_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {knn_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {knn_mae:.2f}")
    print(f"R-squared (R²): {knn_r2:.2f}")

    # Create and fit a Random Forest Regressor model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the model
    rf_y_pred = rf_model.predict(X_test)

    # Calculate evaluation metrics
    rf_mse = mean_squared_error(y_test, rf_y_pred)
    rf_mae = mean_absolute_error(y_test, rf_y_pred)
    rf_r2 = r2_score(y_test, rf_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {rf_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {rf_mae:.2f}")
    print(f"R-squared (R²): {rf_r2:.2f}")


    # Predict missing values using the model (choose the better model based on R²)
    X_missing = df_missing[feature_columns]
    if lr_r2 > knn_r2 and lr_r2 > rf_r2 :
        print("Using Linear Regression for imputation.")
        predicted_values = lr_model.predict(X_missing)
    elif knn_r2 > lr_r2 and knn_r2 > rf_r2:
        print("Using KNN for imputation.")
        predicted_values = knn_model.predict(X_missing)
    else:
        print("Using RF for imputation.")
        predicted_values = rf_model.predict(X_missing)

    # Create a dataset with imputed missing values
    df_filled = df.copy()
    df_filled.loc[df_missing.index, target_column] = predicted_values

    return df_filled, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2


In [24]:
target_columns_to_impute = ['Hours']
feature_columns_for_imputation = ['Year', 'Month', 'DEP', 'Semester']
linear_data = Dinning.copy()


In [25]:
for target_column in target_columns_to_impute:
    linear_data, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2 = impute_with_models(linear_data, target_column, feature_columns_for_imputation)
    print(f"Linear Regression - MSE: {lr_mse}, MAE: {lr_mae}, R²: {lr_r2}")
    print(f"KNN - MSE: {knn_mse}, MAE: {knn_mae}, R²: {knn_r2}")
    print(f"RF - MSE: {rf_mse}, MAE: {rf_mae}, R²: {rf_r2}")

Linear Regression Evaluation metrics for Hours:
Mean Squared Error (MSE): 93124.03
Mean Absolute Error (MAE): 240.15
R-squared (R²): 0.53
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 38851.26
Mean Absolute Error (MAE): 142.47
R-squared (R²): 0.80
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 36142.69
Mean Absolute Error (MAE): 118.35
R-squared (R²): 0.82
Using RF for imputation.
Linear Regression - MSE: 93124.03487406176, MAE: 240.1524277249985, R²: 0.5296154154760799
KNN - MSE: 38851.2645644429, MAE: 142.47411440000002, R²: 0.8037559694971443
RF - MSE: 36142.6881077723, MAE: 118.35378813252049, R²: 0.8174374279191864


In [10]:
linear_data

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [11]:
missing_values = linear_data.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [14]:
df_2022 = linear_data[linear_data['Year'] == 2022]
df_2022


,DEP,Semester,Month,Year,Hours,Money Spent
16,1,2,1,2022,2294.347333,0.000000
17,1,3,2,2022,2226.052500,0.000000
18,1,3,3,2022,2350.855833,0.000000
19,1,3,4,2022,1921.993833,0.000000
20,1,3,5,2022,1308.036833,0.000000
21,1,4,6,2022,509.996167,0.000000
22,1,4,7,2022,474.740167,0.000000
23,1,4,8,2022,644.762500,0.000000
24,1,1,9,2022,2654.713333,0.000000
25,1,1,10,2022,2358.060167,0.000000


In [60]:
df_2022 = linear_data[linear_data['Year'] == 2022]
df_2022

,DEP,Semester,Month,Year,Hours,Money Spent
16,1,2,1,2022,2256.261700,0.000000
17,1,3,2,2022,2181.920400,0.000000
18,1,3,3,2022,2321.431600,0.000000
19,1,3,4,2022,1911.960200,0.000000
20,1,3,5,2022,1325.833600,0.000000
21,1,4,6,2022,485.203200,0.000000
22,1,4,7,2022,467.473300,0.000000
23,1,4,8,2022,623.618000,0.000000
24,1,1,9,2022,2608.653900,0.000000
25,1,1,10,2022,2368.778400,0.000000


In [15]:
CleanedDinningData = linear_data.copy()


In [16]:
CleanedDinningData

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [18]:
CleanedDinningData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          240 non-null    int64  
 1   Semester     240 non-null    int64  
 2   Month        240 non-null    int64  
 3   Year         240 non-null    int64  
 4   Hours        240 non-null    float64
 5   Money Spent  240 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 11.4 KB


In [19]:
missing_values = CleanedDinningData.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64